#Drawing and Bitwise Operations

The next series of lessons covers a basic toolkit of skimage operators. With these tools, we will be able to create programs to perform simple analyses of images based on changes in color or shape. 

We may need to create a *mask* to select only certain parts of an image. In this lesson we explore some tools for doing this.

##Drawing on images

Often we wish to select only a portion of an image to analyze, and ignore the rest. Creating a rectangular sub-image with slicing, as we did in the previous lesson is one option for simple cases. Another option is to create another special image, of the same size as the original, with black pixels indicating the region to save and white pixels everywhere else. Such an image is called a *mask*. In preparing a mask, we sometimes need to be able to draw a black shape – a circle or a rectangle, say – on a white image. skimage provides tools to do that.

Consider this image of maize seedlings:

<img src="https://datacarpentry.org/image-processing/fig/03-maize-roots.jpg" alt="maize seedlings" style="float: left; margin-right:10px;"/>

Now, suppose we want to analyze only the area of the image containing the roots themselves; we do not care to look at the kernels, or anything else about the plants. Further, we wish to exclude the frame of the container holding the seedlings as well. Exploration with ImageJ could tell us that the upper-left coordinate of the sub-area we are interested in is $\left(44, 357\right)$, while the lower-right coordinate is $\left(720, 740\right)$. These coordinates are shown in $\left(x, y\right)$ order.

A Python program to create a mask to select only that area of the image would start with a now-familiar section of code to open and display the original image. (Note that the display portion is used here for pedagogical purposes; it would probably not be used in production code.)

In [ ]:
# one-time imports and setup for displaying images in Colab
import skimage.io

from matplotlib import pyplot as plt

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

import numpy as np

In [ ]:
# load and display the maize roots image
image = skimage.io.imread('https://i.imgur.com/gpLmzNk.png')
skimage.io.imshow(image)
plt.show()

As before, we first import `skimage.io`, and prepare the environment for displaying our images with `matplotlib`. We also import the NumPy library, and give it an alias of `np`. NumPy is necessary when we create the initial mask image, and the alias saves us a little typing. Then, we load and display the initial image in the same way we have done before.

NumPy allows indexing of images/arrays with "boolean" arrays of the same size. Indexing with a boolean array is also called mask indexing. The "pixels" in such a mask array can only take two values: `True` or `False`. If we view an image with pixels set to `True` or `False`, we will see the areas that are `True` as white, and the areas that are `False` as black. 

We can use such a mask to select certain parts of an image: the pixels that correspond to `True` pixels in the mask are selected, while the pixels that correspond to `False` in the mask are not. 

To generate a mask array of the same size as the image, we can use the NumPy module. The next section of code shows how.

In [ ]:
# Create the basic, all True / white, mask
mask = np.ones(shape=image.shape[0:2], dtype="bool")

We create the mask image with the

```
mask = np.ones(shape=image.shape[0:2], dtype="bool")
```

function call. The first argument to the `ones()` function is the shape of the original image, so that our mask will be exactly the same size as the original. Notice that we have only used the first two indices of our shape. We omitted the channel dimension. Indexing with such a mask will change all channel values simultaneously. 

The second argument, `dtype = "bool"`, indicates that the elements in the array should be booleans – i.e., values are either `True` or `False`. Thus, even though we use `np.ones()` to create the mask, its pixel values are in fact not `1` but `True`. You could check this, e.g., by `print(mask[0, 0])`.

Next, we draw a filled, black rectangle on the mask, by setting a slice of the mask to `False` rather than `True`. 

In [ ]:
# Draw filled rectangle on the mask image
import skimage.draw
rr, cc = skimage.draw.rectangle(start=(357, 44), end=(740, 720))
mask[rr, cc] = False

Drawing in this way is a two-step process. First, the `skimage.draw.rectangle()` function is used to create the set of rows and columns (`rr` and `cc`) that will be in the rectangle. Then, we use a special NumPy indexing syntax to make all of the pixels in the selected range a certain color (in this case, `False`). 

Here is what our constructed mask looks like:

In [ ]:
skimage.io.imshow(mask)
plt.show()

---
> **Other drawing operations**
> 
> There are other functions for drawing on images, in addition to the 
> `skimage.draw.rectangle()` function. We can draw circles, lines, text, and 
> other shapes as well. These drawing functions may be useful later on, to help 
> annotate images that our programs produce. Practice some of these functions 
> here. Modify the following code cell, which creates a black, $800 \times 600$
> pixel image. Your task is to draw some other colored shapes and lines on 
> the image, perhaps something like this:
> 
> <img src="https://datacarpentry.org/image-processing/fig/03-draw-practice.jpg" alt="drawing practice" style="float: left; margin-right:10px;"/>
>
> Circles can be drawn with the `skimage.draw.circle()` function, which takes 
> three parameters: the $x$ and $y$ coordinate of the center of the circle, and 
> the radius of the filled circle. There is an optional shape parameter that 
> can be supplied to this function. It will limit the output coordinates for 
> cases where the circle dimensions exceed the ones of the image.
> 
> Lines can be drawn with the `skimage.draw.line()` function, which takes four 
> parameters: the $x$ and $y$ coordinate of one end of the segment, and the $x$ 
> and $y$ coordinate of the other end of the segment.
> 
> Other drawing functions supported by skimage can be found in the <a href="https://scikit-image.org/docs/dev/api/skimage.draw.html?highlight=draw#module-skimage.draw">skimage reference pages</a>.
---

In [ ]:
# TODO: write your drawing code here


##Image modification

All that remains is the task of modifying the image using our mask in such a way that the areas with `True` / white pixels in the mask are not shown in the image any more.

We can use our mask to select all the pixels we are not interested in -- the ones that correspond to `True` in the mask. Then, we can set these pixels to zero, effectively turning them off, while leaving the pixels we are interested in unchanged. 

This code cell demonstrates the whole process of selecting only the seedling roots. 

In [ ]:
# load original image
image = skimage.io.imread('https://i.imgur.com/gpLmzNk.png')

# create the mask
rr, cc = skimage.draw.rectangle(start=(357, 44), end=(740, 720))
mask[rr, cc] = False

# apply the mask
image[mask] = 0

# display the result
skimage.io.imshow(image)
plt.show()

---
> **Masking an image of your own**
> 
> Now, it is your turn to pracice. Upload an image of an object with a simple 
> overall geometric shape (think rectangular or circular). Write code like the 
> sample above to select only the primary object in your image. For example, 
> here is an image of a remote control:
> 
> <img src="https://datacarpentry.org/image-processing/fig/03-remote-control.jpg" alt="remote control before" style="float: left; margin-right:10px;"/>
> 
> And, here is the end result of code that masks out (almost) everything
> but the remote:
>
> <img src="https://datacarpentry.org/image-processing/fig/03-remote-control-masked.jpg" alt="remote control masked" style="float: left; margin-right:10px;"/>
---

In [ ]:
# TODO: write your masking code here


---
> **Masking a 96-well plate image**
> 
> Consider this image of a 96-well plate that was scanned on a flatbed scanner.
>
> <img src="https://datacarpentry.org/image-processing/fig/03-wellplate.jpg" alt="96-well plate" style="float: left; margin-right:10px;"/>
>
> Suppose that we are interested in the colors of the solutions in each of the 
> wells. We do not care about the color of the rest of the image, i.e., the 
> plastic that makes up the well plate itself. The URL for this image is 
> https://i.imgur.com/QtYg6Zi.png.
> 
> Execute the code cell below. This will load a file named `centers.txt`
> from the cloud. This file contains the $\left(x, y\right)$ coordinates
> of the center of each of the 96 wells on the plate, with one pair per 
> line. You may assume that each of the wells in the image has a radius
> of 16 pixels. In the second code cell below, write Python to read in the
> well plate image, and the `centers.txt` file, and then mask out everything
> we are not interested in studying from the image. Your program sould 
> produce output that looks like this:
>
> <img src="https://datacarpentry.org/image-processing/fig/03-wellplate-masked.jpg" alt="masked well plate" style="float: left; margin-right:10px;"/>
---

In [ ]:
# execute this cell to download centers.txt
!curl 'https://raw.githubusercontent.com/mmeysenburg/ist371/main/centers.txt' > centers.txt

In [ ]:
# TODO write your masking code here


---
> **Masking a 96-well plate image, take two**
> 
> If you spent some time looking at the contents of the `centers.txt` file 
> from the previous challenge, you may have noticed that the centers of 
> each well in the image are very regular. Assuming that the images are 
> scanned in such a way that the wells are always in the same place, and 
> that the image is perfectly oriented (i.e., it does not slant one way or 
> another), we could produce our well plate mask without having to read in 
> the coordinates of the centers of each well. Assume that the center of 
> the upper left well in the image is at location $x = 91$ and $y = 108$, 
> and that there are 70 pixels between each center in the $x$ dimension and 
> 72 pixels between each center in the $y$ dimension. Each well still has a 
> radius of 16 pixels. In the code cell below, write Python code that 
> produces the same output image as in the previous challenge, but without 
> having to read in the `centers.txt` file. *Hint: use nested for loops.*
---

In [ ]:
# TODO: write masking code here
